# Въведение

Бях чел тази [статия](http://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html) преди да говорим за `LDA` на лекциите ("Latent Dirichlet Allocation"). Още не го разбирам, че вътре математиката ми е над възможностите, ама нека се опитаме да го използваме, за да видим какъв резултат ще постигнем.

## Какво е `LDA` накратко?

*Latent Dirichlet allocation (LDA) is a topic model that generates topics based on word frequency from a set of documents. LDA is particularly useful for finding reasonably accurate mixtures of topics within a given document set.*

**TL;DR Ползваме го, за да разберем за какво е текста.** Той намира някакви общи зависимости между думите, като така изкарва темата/темите на съответния текст.

Искам да го пробвам, защото авторите най-вероятно имат общи теми в своите текстове. Освен това видяхме, че са живели в горе-долу различни епохи, което също може да афектира стила и темите им на писане. Но вместо да гадаем, ето какво казва Уикипедия:

- [Едгар Алън По](https://bg.wikipedia.org/wiki/%D0%95%D0%B4%D0%B3%D0%B0%D1%80_%D0%90%D0%BB%D1%8A%D0%BD_%D0%9F%D0%BE#.D0.9B.D0.B8.D1.82.D0.B5.D1.80.D0.B0.D1.82.D1.83.D1.80.D0.B5.D0.BD_.D1.81.D1.82.D0.B8.D0.BB_.D0.B8_.D1.82.D0.B5.D0.BC.D0.B0.D1.82.D0.B8.D0.BA.D0.B0)
- [Хауърд Филипс Лъвкрафт](https://bg.wikipedia.org/wiki/%D0%A5%D0%B0%D1%83%D1%8A%D1%80%D0%B4_%D0%9B%D1%8A%D0%B2%D0%BA%D1%80%D0%B0%D1%84%D1%82#.D0.91.D0.B8.D0.B1.D0.BB.D0.B8.D0.BE.D0.B3.D1.80.D0.B0.D1.84.D0.B8.D1.8F)
- [Мери Уолстонкрафт Шели](https://bg.wikipedia.org/wiki/%D0%9C%D0%B5%D1%80%D0%B8_%D0%A8%D0%B5%D0%BB%D0%B8#.D0.A2.D0.B2.D0.BE.D1.80.D0.B1.D0.B8_.D0.BD.D0.B0_.D0.9C.D0.B5.D1.80.D0.B8_.D0.A8.D0.B5.D0.BB.D0.B8)

### Или с 2 думи:

- Едгар Алън По - смъртта, включително физически знаци, ефектите от разлагането, преждевременното погребение, съживяване на мъртвите и скръбта
- Хауърд Филипс Лъвкрафт - Ктхулу
- Мери Уолстонкрафт Шели - Франкенщайн


![wtf](https://i.giphy.com/media/V1gFqXnfRpVxS/giphy.webp)
*Нормални автори нямаше ли!?*

Да си заредим данните първо.

In [2]:
import pandas as pd
train = pd.read_csv("data/train.zip", index_col=['id'])
test = pd.read_csv("data/test.zip", index_col=['id'])
sample_submission = pd.read_csv("data/sample_submission.zip", index_col=['id'])

print(train.shape, test.shape, sample_submission.shape)
print(set(train.columns) - set(test.columns))

(19579, 2) (8392, 1) (8392, 3)
{'author'}


# Нека си направим план

Какво да направим, че да се справим оптимално добре?

Първо, трябва да отбележим, че `LDA` минава доста бавно и трябва внимателно да подбираме стъпките, които правим (че в момента на писане на домашното е събота и имам далеч по-приятни неща за "чакане" :D)

## План
1. Нека лематизираме (`lemmatization`) думите в текста първо. По време на лекцията ползвахме "стематизация" (`stemming`), та реших да вкарам нещо ново. *Тези готини думи едва ли се пишат така на български...*
2. Да махнем `stop-words`
3. `LDA` в scikit-learn очаква да му подадем вектор от думите, ама да сме ги минали първо през `CountVectorizer` или `TfidfVectorizer` (или някакъв друг), за да може да работи. Това е добре защото, направо можем да си `Pipeline`-нем плана на действие. Ако сте видели статията от по-горе (което не сте направили) сте видели, че използват друга библиоетка (`genism`), където `LDA` очаква да му се подаде списък със самите думи от документа. *Бях отделил около половин ден, за да докарам такъв тип данни до `LDA` и да го подкарам в scikit-learn, но естествено нямаше как да стане. "Четете си документацията!" :(*
4. Целта на предните ни 2 точки беше да направим т.нар. "bag-of-words". С други думи на `LDA` му трябва някой да му "токенизира" думите. Когато вече имаме вектора от текста можем спокойно да използваме `LatentDirichletAllocation`
5. Да изсипем всички това към някакъв класификатор. Ще пробваме с `LogisticRegression` или някакво `SVC`. Накрая ще пробвам и с `MultinomialNB`, както направихме в лекцията, че поради някаква причина naive Bayes класификаториите най-добре се справят с текстове (или поне така Лъчо и някакви хора в SO казват).

Ето една [статия](http://sebastianraschka.com/Articles/2014_naive_bayes_1.html), която описва детайлно NB.

Нека започнем да изпълняваме стъпките. Ще си направим една функция, която прави стъпки 1. и 2. Нещо такова:

In [3]:
def prepare(text):
    def remove_stop_words(tokens):
        pass
    
    def lemmatize():
        pass

За да махнем "стоп-думите" ще изпозлваме [тази библиотека](https://pypi.python.org/pypi/stop-words)

In [4]:
from stop_words import get_stop_words


def prepare(df):
    def remove_stop_words(tokens):
        english_stop_words = get_stop_words('en')

        return [token for token in tokens if token not in english_stop_words]

    def lemmatize():
        pass

Сега лематизираме. Първо ще го пробвам да видим какво става, че досега не сме го използвали.

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/martin056/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text = train.iloc[0]['text']

[lemmatizer.lemmatize(token) for token in text.split()]

['This',
 'process,',
 'however,',
 'afforded',
 'me',
 'no',
 'mean',
 'of',
 'ascertaining',
 'the',
 'dimension',
 'of',
 'my',
 'dungeon;',
 'a',
 'I',
 'might',
 'make',
 'it',
 'circuit,',
 'and',
 'return',
 'to',
 'the',
 'point',
 'whence',
 'I',
 'set',
 'out,',
 'without',
 'being',
 'aware',
 'of',
 'the',
 'fact;',
 'so',
 'perfectly',
 'uniform',
 'seemed',
 'the',
 'wall.']

## Супер

Само че открихме няколко проблема:
  - `.lower()` на думите
  - сами трябва да се оправим с разликата между глаголи и съществителни ("meeting" например).
  - `.split()` не е чак толкова умно и не се оправя с препинателни знаци


### Решения
- Първият проблем е ясен
- За втория открих нещо наречено `pos_tag` в `nltk` и ще се опитаме да го използваме
- За третия - `RegexpTokenizer`

Да видим какво ще ни даде `pos_tag` с примера от преди малко.

In [7]:
import nltk

text = train.iloc[0]['text']

nltk.pos_tag([lemmatizer.lemmatize(token) for token in text.split()])

[('This', 'DT'),
 ('process,', 'NN'),
 ('however,', 'NN'),
 ('afforded', 'VBD'),
 ('me', 'PRP'),
 ('no', 'DT'),
 ('mean', 'NN'),
 ('of', 'IN'),
 ('ascertaining', 'VBG'),
 ('the', 'DT'),
 ('dimension', 'NN'),
 ('of', 'IN'),
 ('my', 'PRP$'),
 ('dungeon;', 'NN'),
 ('a', 'DT'),
 ('I', 'PRP'),
 ('might', 'MD'),
 ('make', 'VB'),
 ('it', 'PRP'),
 ('circuit,', 'JJ'),
 ('and', 'CC'),
 ('return', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('point', 'NN'),
 ('whence', 'NN'),
 ('I', 'PRP'),
 ('set', 'VBP'),
 ('out,', 'RB'),
 ('without', 'IN'),
 ('being', 'VBG'),
 ('aware', 'JJ'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('fact;', 'NNS'),
 ('so', 'RB'),
 ('perfectly', 'RB'),
 ('uniform', 'JJ'),
 ('seemed', 'VBD'),
 ('the', 'DT'),
 ('wall.', 'NN')]

![Браво nltk](https://i.giphy.com/media/8RxCFgu88jUbe/giphy.webp)

Мерси много, `nltk`! Порових се малко в SO и ето решението...

In [8]:
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
tokens_and_tags = nltk.pos_tag([lemmatizer.lemmatize(token).lower() for token in text.split()])

for token, tag in tokens_and_tags:
    print(lemmatizer.lemmatize(token, get_wordnet_pos(tag)))

this
process,
however,
afford
me
no
mean
of
ascertain
the
dimension
of
my
dungeon;
a
i
might
make
it
circuit,
and
return
to
the
point
whence
i
set
out,
without
be
aware
of
the
fact;
so
perfectly
uniform
seem
the
wall.


Тези препинателни знаци, малко ме притесняват. Дайте да ги махнем!

In [9]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"'?\b[0-9A-Za-z'-]+\b'?")  # matches words and words with apostrophes

tokens1 = tokenizer.tokenize(train.iloc[0]['text'])
tokens2 = tokenizer.tokenize(train.iloc[100]['text'])

print(tokens1)
print(tokens2)

['This', 'process', 'however', 'afforded', 'me', 'no', 'means', 'of', 'ascertaining', 'the', 'dimensions', 'of', 'my', 'dungeon', 'as', 'I', 'might', 'make', 'its', 'circuit', 'and', 'return', 'to', 'the', 'point', 'whence', 'I', 'set', 'out', 'without', 'being', 'aware', 'of', 'the', 'fact', 'so', 'perfectly', 'uniform', 'seemed', 'the', 'wall']
['They', 'still', 'appeared', 'in', 'public', 'together', 'and', 'lived', 'under', 'the', 'same', 'roof']


## Сега сме щастливи

Време е да комбинираме всичко задно!

In [10]:
from stop_words import get_stop_words

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer


def prepare(text):
    def tokenize():
        return tokenizer.tokenize(text)

    def remove_stop_words(tokens):
        english_stop_words = get_stop_words('en')

        return [token for token in tokens if token not in english_stop_words]

    def lemmatize(tokens):
        lemmatizer = nltk.stem.WordNetLemmatizer()
        tokens_and_tags = nltk.pos_tag(tokens)
        
        lemmas = []

        for token, tag in tokens_and_tags:
            lemma = lemmatizer.lemmatize(token.lower(), get_wordnet_pos(tag))
            lemmas.append(lemma)
            
        return lemmas
    
    tokens = tokenize()
    tokens = remove_stop_words(tokens)
    
    lemmas = lemmatize(tokens)
    
    return " ".join(lemmas)

## Хайде да използваме нашата красива функция

In [11]:
new_train = train.copy()

new_train['processed_text'] = new_train.apply(lambda r: prepare(r['text']), axis=1)

new_train.head(10)

,text,author,processed_text
id,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afford mean ascertain dim...
id17569,It never once occurred to me that the fumbling...,HPL,it never occur fumble might mere mistake
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in left hand gold snuff box caper hill cut man...
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely spring a looked windsor terrace six...
id12958,"Finding nothing else, not even gold, the Super...",HPL,find nothing else even gold superintendent aba...
id22965,"A youth passed in solitude, my best years spen...",MWS,a youth pass solitude best year spend gentle f...
id09674,"The astronomer, perhaps, at this point, took r...",EAP,the astronomer perhaps point take refuge sugge...
id13515,The surcingle hung in ribands from my body.,EAP,the surcingle hung ribands body
id19322,I knew that you could not say to yourself 'ste...,EAP,i know say 'stereotomy' without bring think at...


![](https://i.giphy.com/media/mHEes6Quf8XK0/giphy.webp)

## Време е да се гмурнем в по-дълбокото ...

Първо да видим какъв резултат ще изкараме без да използваме `LDA` (демек **baseline model**). Ще използваме `CountVectorizer` + `LinearSVC`.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score


pipeline = Pipeline([
    ('features', CountVectorizer()),
    ('clf', LinearSVC())
])

cross_val_score(pipeline, new_train.processed_text, train.author, cv=3, n_jobs=3)

array([ 0.77619485,  0.78455409,  0.78252874])

## Да си поиграем малко

Искам да видя какво се случва, ако използвам различни класификатори - ей така, за идеята, а и после ще ми е по-леско, когато напасвам хиперпараметрите. Ще ползваме и `neg_log_loss`, защото такова е заданието. **Хубаво е всички класификатори, които ползваме да имат `predict_proba()`, че иначе ще трябва да я пишем сами..**

In [17]:
# Imports
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [27]:
for vec in [CountVectorizer, TfidfVectorizer]:
    for clf in [MultinomialNB, LogisticRegression, RandomForestClassifier]:

        pipeline = Pipeline([
            ('vec', vec()),
            ('clf', clf())
        ])

        score_arr = cross_val_score(pipeline,
                                    new_train.processed_text,
                                    train.author,
                                    cv=3, 
                                    n_jobs=3,
                                    scoring='neg_log_loss')

        print(score_arr)

        msg = "{vec} + {clf} scored: {score} with std: {std}".format(vec=vec.__name__,
                                                                     clf=clf.__name__,
                                                                     score=score_arr.mean(),
                                                                     std=score_arr.std())
        print(msg)


[-0.47719056 -0.46267412 -0.45622925]
CountVectorizer + MultinomialNB scored: -0.46536464628356417 with std: 0.008766350399386022
[-0.50442829 -0.49519917 -0.48903599]
CountVectorizer + LogisticRegression scored: -0.49622114930192 with std: 0.0063252942089223226
[-1.52406287 -1.64368489 -1.60907883]
CountVectorizer + RandomForestClassifier scored: -1.592275530321652 with std: 0.0502601243228477
[-0.6064134  -0.60352745 -0.60022146]
TfidfVectorizer + MultinomialNB scored: -0.6033874375047138 with std: 0.002529784953187279
[-0.6229566  -0.6184584  -0.61262533]
TfidfVectorizer + LogisticRegression scored: -0.6180134462785358 with std: 0.0042294424652843925
[-1.61736905 -1.64970077 -1.49802899]
TfidfVectorizer + RandomForestClassifier scored: -1.588366270859846 with std: 0.06522756556756859


**SVC  scored:  -0.766141513098  witf std:  0.0109908929665**
Това е и резултатът на `SVC`-то + `CountVecotrizer`, ама на него му трябва `probability=True` при инициализацията, а мина за 7 минути и не искам да го пускам пак.


## Да се опитаме да анализираме резултатите

Очевидно имаме победител - `MultinomialNB` + `CountVectorizer`.

Добре, за `NB` беше очаквано, понеже работим с текстове. Но защо `CountVectorizer` се справи по-добре от `TfidfVectorizer`? На лекцията изпозлвахме именно `Tfidf`.

Ще продължа да използвам и двете, за да видя какво ще излезе. Но този път с подобрение на хиперпараметрите.

In [13]:
import numpy as np
from pprint import pprint

def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters:")
            pprint(results['params'][candidate])
            print("\n")

In [33]:
from sklearn.model_selection import RandomizedSearchCV


params_count_word = {
    "vec__ngram_range": [(1,1), (1,2), (1,3)],
    "vec__analyzer": ['word'],
    "vec__max_df":[1.0, 0.9, 0.8, 0.7, 0.6, 0.5],
    "vec__min_df":[2, 3, 5, 10]
}

params = {
    "clf__alpha": [0.01, 0.1, 0.5, 1, 2]
}


params.update(params_count_word)

for vec in [CountVectorizer, TfidfVectorizer]:
    pipeline = Pipeline([
        ('vec', vec()),
        ('clf', MultinomialNB())
    ])
    
    print("Using {}".format(vec.__name__))
    random_search = RandomizedSearchCV(pipeline, param_distributions=params, 
                                       scoring='neg_log_loss',
                                       n_iter=20, cv=3, n_jobs=4)

    random_search.fit(new_train.processed_text, train.author)
    report(random_search.cv_results_)

Using CountVectorizer
Model with rank: 1
Mean validation score: -0.479 (std: 0.010)
Parameters:
{'clf__alpha': 2,
 'vec__analyzer': 'word',
 'vec__max_df': 0.5,
 'vec__min_df': 3,
 'vec__ngram_range': (1, 2)}


Model with rank: 1
Mean validation score: -0.479 (std: 0.010)
Parameters:
{'clf__alpha': 2,
 'vec__analyzer': 'word',
 'vec__max_df': 1.0,
 'vec__min_df': 3,
 'vec__ngram_range': (1, 2)}


Model with rank: 3
Mean validation score: -0.480 (std: 0.011)
Parameters:
{'clf__alpha': 1,
 'vec__analyzer': 'word',
 'vec__max_df': 0.6,
 'vec__min_df': 2,
 'vec__ngram_range': (1, 3)}


Model with rank: 4
Mean validation score: -0.488 (std: 0.011)
Parameters:
{'clf__alpha': 1,
 'vec__analyzer': 'word',
 'vec__max_df': 0.6,
 'vec__min_df': 3,
 'vec__ngram_range': (1, 2)}


Model with rank: 5
Mean validation score: -0.498 (std: 0.007)
Parameters:
{'clf__alpha': 2,
 'vec__analyzer': 'word',
 'vec__max_df': 0.8,
 'vec__min_df': 5,
 'vec__ngram_range': (1, 1)}


Model with rank: 5
Mean validatio

Резултатите ни са най-добри до горната граница (2). Ще пробвам с малко по-големи.

In [36]:
params_count_word = {
    "vec__ngram_range": [(1,1), (1,2), (1,3)],
    "vec__analyzer": ['word'],
    "vec__max_df":[1.0, 0.9, 0.8, 0.7, 0.6, 0.5],
    "vec__min_df":[2, 3, 5, 10]
}

params = {
    "clf__alpha": range(2, 11)
}


params.update(params_count_word)

def search():
    pipeline = Pipeline([
        ('vec', CountVectorizer()),
        ('clf', MultinomialNB())
    ])
    
    random_search = RandomizedSearchCV(pipeline, param_distributions=params, 
                                       scoring='neg_log_loss',
                                       n_iter=20, cv=3, n_jobs=4)

    random_search.fit(new_train.processed_text, train.author)
    report(random_search.cv_results_)
    

search()

Model with rank: 1
Mean validation score: -0.478 (std: 0.008)
Parameters:
{'clf__alpha': 2,
 'vec__analyzer': 'word',
 'vec__max_df': 0.8,
 'vec__min_df': 3,
 'vec__ngram_range': (1, 1)}


Model with rank: 2
Mean validation score: -0.480 (std: 0.007)
Parameters:
{'clf__alpha': 4,
 'vec__analyzer': 'word',
 'vec__max_df': 0.9,
 'vec__min_df': 2,
 'vec__ngram_range': (1, 1)}


Model with rank: 3
Mean validation score: -0.483 (std: 0.008)
Parameters:
{'clf__alpha': 5,
 'vec__analyzer': 'word',
 'vec__max_df': 0.5,
 'vec__min_df': 2,
 'vec__ngram_range': (1, 3)}


Model with rank: 4
Mean validation score: -0.498 (std: 0.008)
Parameters:
{'clf__alpha': 3,
 'vec__analyzer': 'word',
 'vec__max_df': 0.5,
 'vec__min_df': 5,
 'vec__ngram_range': (1, 2)}


Model with rank: 5
Mean validation score: -0.499 (std: 0.008)
Parameters:
{'clf__alpha': 4,
 'vec__analyzer': 'word',
 'vec__max_df': 0.6,
 'vec__min_df': 5,
 'vec__ngram_range': (1, 2)}




Подбрахме някакви хиперпараметри и имаме обща идея какво можем да променим, за да си подобрим оценката.

Време е да вкараме `LDA`. Първо без никакви хиперпараметри.

In [37]:
from sklearn.decomposition import LatentDirichletAllocation

pipeline = Pipeline([
    ('vec', CountVectorizer()),
    ('lda', LatentDirichletAllocation()),
    ('clf', MultinomialNB())
])

score_arr = cross_val_score(pipeline,
                        new_train.processed_text,
                        train.author,
                        cv=3, 
                        n_jobs=3,
                        scoring='neg_log_loss')

print(score_arr)
print("mean: ", score_arr.mean())
print("std: ", score_arr.std())

/home/martin056/.virtualenvs/machine-learning/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/martin056/.virtualenvs/machine-learning/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/martin056/.virtualenvs/machine-learning/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


[-1.06364151 -1.06972602 -1.07343668]
mean:  -1.06893473718
std:  0.00403781324269


Това е доста зле...

Ще пробвам направо с оригиналните текстове - в лекцията видяхме, че това подобри резултата, макар че ме съмнява да помогне в случая.

In [18]:
from sklearn.decomposition import LatentDirichletAllocation

pipeline = Pipeline([
    ('vec', CountVectorizer()),
    ('lda', LatentDirichletAllocation()),
    ('clf', MultinomialNB())
])

score_arr = cross_val_score(pipeline,
                        new_train.text,
                        train.author,
                        cv=3, 
                        n_jobs=3,
                        scoring='neg_log_loss')

print(score_arr)
print("mean: ", score_arr.mean())
print("std: ", score_arr.std())

/home/martin056/.virtualenvs/machine-learning/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/martin056/.virtualenvs/machine-learning/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/martin056/.virtualenvs/machine-learning/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


[-1.07466632 -1.06065831 -1.07202835]
mean:  -1.06911766055
std:  0.00607783817284


## Не се получава...

Нека поне видим какви теми намира `LDA`.

Но преди това ще променя `learning_method`, за да премахнем този warning (а и може това да подобри лошия ни score)

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

pipeline = Pipeline([
    ('vec', CountVectorizer()),
    ('lda', LatentDirichletAllocation(learning_method='batch')),
    ('clf', MultinomialNB())
])

score_arr = cross_val_score(pipeline,
                            new_train.processed_text,
                            train.author,
                            cv=3,
                            n_jobs=3,
                            scoring='neg_log_loss')

print(score_arr)
print("mean: ", score_arr.mean())
print("std: ", score_arr.std())

[-1.05114669 -1.01765627 -1.01981837]
mean:  -1.02954044022
std:  0.0153034016344


А сега да визуализираме темите.

In [25]:
import mglearn

vectorizer = CountVectorizer(max_df=0.5, max_features=10000)
X = vectorizer.fit_transform(new_train.processed_text)

lda = LatentDirichletAllocation(n_components=10, learning_method="batch", max_iter=15, random_state=0)
topics = lda.fit_transform(X)

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vectorizer.get_feature_names())
mglearn.tools.print_topics(topics=range(10), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
will          heart         door          the           see           
upon          voice         room          upon          like          
the           eye           the           one           it            
say           now           house         foot          far           
thus          one           open          street        come          
make          love          find          two           eye           
may           word          it            wall          earth         
however       death         now           three         still         
give          upon          within        light         look          
mr            yet           thing         side          star          


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
hand

## Поне намерихме проблема

Аз лично не мога да направя ясно разграничение между темите. Очаквах да има думи, като Франкенщайн или Ктхулу, което ясно щеше да покаже кой е авторът.

За жалост такива думи в теми няма. Очевидно е, че `LDA` намира теми, но според мен те са близки за всички автори.

Като се загледаме малко повече, всъщност можем да видим, че в тема 7 има 2 имена - `Raymond` и `Perdita`. Google ми подсказа, че това са герои от новелата `The Last Man` на Мери Шели. Също така, сещайки се за историята за Франкенщайн, можем да предполагаме, че тема 1 е именно за него, коет отново ни води до Мери Шели.

От останалите теми не мисля, че можем да разберем нещо (а и не съм запознат с творчеството на авторите).

`LDA` не ни помогна в подобряването на резултата, но поне опитахме...